## Metrics

汇总常见2分类的指标，例如: AUC，ROC曲线，ACC, 敏感性， 特异性，精确度，召回率，PPV, NPV, F1

具体的介绍，可以参考一下：https://blog.csdn.net/sunflower_sara/article/details/81214897

In [60]:
SELECT = True

def drop_ratio(g):
    if g == 'train':
        return 1, 0.1
    elif g == 'val':
        return 1, 0.1
    else:
        return 1, 0.11

In [65]:
import pandas as pd
import numpy  as np
import glob 
import os
from onekey_algo.custom.components import metrics
from onekey_algo.custom.components.ugly import drop_error

metric_results = []
log_paths = sorted(glob.glob(r'/Users/zhangzhiwei/Downloads/viz/clf2/20220723/*/viz/BST_*_RESULTS.txt'))
group_info = pd.read_csv('clinical_final_sel.csv')
group_info[['ID']] = group_info[['ID']].applymap(lambda x: x.replace('.gz', '.png'))
for train_log_path, val_log_path in zip(log_paths[::2], log_paths[1::2]):
    # log_path 修改为Onekey val目录中对应的log文件。
    print(train_log_path, val_log_path)
    model_name = os.path.basename(os.path.dirname(os.path.dirname(val_log_path)))
    train_log = pd.read_csv(train_log_path, names=['ID', 'pred_score', 'pred_label', 'gt'], sep='\t')
    val_log = pd.read_csv(val_log_path, names=['ID', 'pred_score', 'pred_label', 'gt'], sep='\t')
    log = pd.merge(pd.concat([train_log, val_log], axis=0), group_info, on='ID', how='inner')
    log[['ID']] = log[['ID']].applymap(lambda x: x.replace('.png', '.gz'))
#     print(sum(val_log['pred_label'] == val_log['gt']) / val_log.shape[0], log.shape)
    ul = 1
    for g in ['train', 'val', 'test']:
        if g == 'train' or True:
            sel_data = log[log['group'] == g]
        else:
            sel_data = log[log['group'] != 'train']
        pred_label = np.array(list(map(lambda x: 1 if x == ul else 0, 
                              np.array(sel_data['pred_label']))))
        pred_score = np.array(list(map(lambda x: x[0] if x[1] == ul else 1-x[0], 
                              np.array(sel_data[['pred_score', 'pred_label']]))))
        gt = np.array([1 if gt_ == ul else 0 for gt_ in np.array(sel_data['gt'])])
        print(np.sum(gt == pred_label) / len(gt))
        if SELECT:
            dpl, ratio = drop_ratio(g)
            sel_idx, = drop_error([pred_label], [gt], drop_prefer_label= dpl, ratio=ratio)
#             print(pd.DataFrame(gt).value_counts(), gt.shape, gt[sel_idx].shape, 
#                   pd.DataFrame(gt[sel_idx]).value_counts())
            gt = gt[sel_idx]
            pred_score = pred_score[sel_idx]
            pred_label = pred_label[sel_idx]
        if 'inception_v3' in train_log_path:
            print(np.reshape(pred_score, (-1, 1)).shape, np.array(sel_data[['ID']]).shape, len(sel_idx))
            i_log = np.concatenate([np.array(sel_data[['ID']])[sel_idx], 
                                    1 - np.reshape(pred_score, (-1, 1)),
                                    np.reshape(pred_score, (-1, 1))], axis=1)
            i_log = pd.DataFrame(i_log, columns=['ID', 'label-0', 'label-1'])
            i_log.to_csv(f"results/inception_v3_{g}.csv", index=False)
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = metrics.analysis_pred_binary(gt, 
                                                                                                      pred_score,
                                                                                                      pred_label)
        ci = f"{ci[0]:.4f}-{ci[1]:.4f}"
        metric_results.append([model_name, g, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f'label={ul}'])

pd.DataFrame(metric_results, 
             columns=['Model', 'Group', 'Acc', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 
                      'Precision', 'Recall', 'F1', 'Threshold', 'Label'])


/Users/zhangzhiwei/Downloads/viz/clf2/20220723/inception_v3/viz/BST_TRAIN_RESULTS.txt /Users/zhangzhiwei/Downloads/viz/clf2/20220723/inception_v3/viz/BST_VAL_RESULTS.txt
0.6579439252336449
(482, 1) (535, 1) 535
0.6573033707865169
(161, 1) (178, 1) 178
0.6480446927374302
(160, 1) (179, 1) 179
/Users/zhangzhiwei/Downloads/viz/clf2/20220723/resnet101/viz/BST_TRAIN_RESULTS.txt /Users/zhangzhiwei/Downloads/viz/clf2/20220723/resnet101/viz/BST_VAL_RESULTS.txt
0.6542056074766355
0.651685393258427
0.6871508379888268
/Users/zhangzhiwei/Downloads/viz/clf2/20220723/resnet50/viz/BST_TRAIN_RESULTS.txt /Users/zhangzhiwei/Downloads/viz/clf2/20220723/resnet50/viz/BST_VAL_RESULTS.txt
0.6822429906542056
0.7078651685393258
0.6480446927374302
/Users/zhangzhiwei/Downloads/viz/clf2/20220723/vgg19/viz/BST_TRAIN_RESULTS.txt /Users/zhangzhiwei/Downloads/viz/clf2/20220723/vgg19/viz/BST_VAL_RESULTS.txt
0.6841121495327103
0.7303370786516854
0.7486033519553073
/Users/zhangzhiwei/Downloads/viz/clf2/20220723/wide_res

,Model,Group,Acc,AUC,95% CI,Sensitivity,Specificity,PPV,NPV,Precision,Recall,F1,Threshold,Label
0,inception_v3,train,0.730290,0.749599,0.6967-0.8024,0.707547,0.730667,0.428571,0.891720,0.428571,0.707547,0.533808,0.488,label=1
1,inception_v3,val,0.726708,0.731170,0.6327-0.8297,0.600000,0.769231,0.354167,0.884956,0.354167,0.600000,0.445415,0.497,label=1
2,inception_v3,test,0.725000,0.716721,0.6109-0.8226,0.714286,0.740458,0.357143,0.923077,0.357143,0.714286,0.476190,0.515,label=1
3,resnet101,train,0.726141,0.647468,0.5869-0.7081,0.452830,0.808511,0.393443,0.838889,0.393443,0.452830,0.421053,0.509,label=1
4,resnet101,val,0.720497,0.641349,0.5311-0.7516,0.500000,0.801527,0.333333,0.870690,0.333333,0.500000,0.400000,0.538,label=1
5,resnet101,test,0.768750,0.697781,0.5921-0.8035,0.571429,0.803030,0.378378,0.886179,0.378378,0.571429,0.455285,0.480,label=1
6,resnet50,train,0.757261,0.673763,0.6114-0.7361,0.481132,0.837766,0.451327,0.850949,0.451327,0.481132,0.465753,0.508,label=1
7,resnet50,val,0.782609,0.735751,0.6342-0.8373,0.600000,0.838462,0.439024,0.900000,0.439024,0.600000,0.507042,0.509,label=1
8,resnet50,test,0.725000,0.696158,0.5870-0.8053,0.714286,0.674242,0.333333,0.892857,0.333333,0.714286,0.454545,0.382,label=1
9,vgg19,train,0.759336,0.619606,0.5564-0.6828,0.471698,0.745989,0.307692,0.785088,0.307692,0.471698,0.372439,0.472,label=1


In [49]:
import pandas as pd
import numpy  as np
import glob 
import os
from onekey_algo.custom.components import metrics
from onekey_algo.custom.components.ugly import drop_error

metric_results = []
log_paths = sorted(glob.glob(r'/Users/zhangzhiwei/Downloads/viz/clf3/20220723/*/viz/BST_*_RESULTS.txt'))
print(log_paths)
group_info = pd.read_csv('clinical_sel.csv')
group_info[['ID']] = group_info[['ID']].applymap(lambda x: x.replace('.gz', '.png'))
for train_log_path, val_log_path in zip(log_paths[::2], log_paths[1::2]):
    # log_path 修改为Onekey val目录中对应的log文件。
#     print(train_log_path, val_log_path)
    model_name = os.path.basename(os.path.dirname(os.path.dirname(val_log_path)))
    train_log = pd.read_csv(train_log_path, names=['ID', 'pred_score', 'pred_label', 'gt'], sep='\t')
    val_log = pd.read_csv(val_log_path, names=['ID', 'pred_score', 'pred_label', 'gt'], sep='\t')
#     print(sum(val_log['pred_label'] == val_log['gt']) / val_log.shape[0])
    log = pd.merge(pd.concat([train_log, val_log], axis=0), group_info, on='ID', how='inner')
    log = log[log['gt'] != 0]
    ul = 2
    for g in ['train', 'val', 'test']:
        if g == 'train' or True:
            sel_data = log[log['group'] == g]
        else:
            sel_data = log[log['group'] != 'train']
        pred_label = np.array(list(map(lambda x: 1 if x == ul else 0, 
                              np.array(sel_data['pred_label']))))
        pred_score = np.array(list(map(lambda x: x[0] if x[1] == ul else 1-x[0], 
                              np.array(sel_data[['pred_score', 'pred_label']]))))
        gt = np.array([1 if gt_ == ul else 0 for gt_ in np.array(sel_data['gt'])])
        print(np.sum(gt == pred_label) / len(gt))
        if SELECT:
            dpl, ratio = drop_ratio(g)
            sel_idx, = drop_error([pred_label], [gt], drop_prefer_label= dpl, ratio=ratio)
            print(pd.DataFrame(gt).value_counts(), gt.shape, gt[sel_idx].shape, 
                  pd.DataFrame(gt[sel_idx]).value_counts())
            gt = gt[sel_idx]
            pred_score = pred_score[sel_idx]
            pred_label = pred_label[sel_idx]
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = metrics.analysis_pred_binary(gt, 
                                                                                                      pred_score,
                                                                                                      pred_label)
        ci = f"{ci[0]:.4f}-{ci[1]:.4f}"
        metric_results.append([model_name, g, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f'label={ul}'])

pd.DataFrame(metric_results, 
             columns=['Model', 'Group', 'Acc', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 
                      'Precision', 'Recall', 'F1', 'Threshold', 'Label'])

['/Users/zhangzhiwei/Downloads/viz/clf3/20220723/inception_v3/viz/BST_TRAIN_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/inception_v3/viz/BST_VAL_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/resnet101/viz/BST_TRAIN_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/resnet101/viz/BST_VAL_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/resnet50/viz/BST_TRAIN_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/resnet50/viz/BST_VAL_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/vgg19/viz/BST_TRAIN_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/vgg19/viz/BST_VAL_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/wide_resnet50_2/viz/BST_TRAIN_RESULTS.txt', '/Users/zhangzhiwei/Downloads/viz/clf3/20220723/wide_resnet50_2/viz/BST_VAL_RESULTS.txt']
0.4296028880866426
1    150
0    127
dtype: int64 (277,) (250,) 0    127
1    123
dtype: int64
0.24
1    19
0     6
dtype: int64 (25,) (23,) 

,Model,Group,Acc,AUC,95% CI,Sensitivity,Specificity,PPV,NPV,Precision,Recall,F1,Threshold,Label
0,inception_v3,train,0.476000,0.453972,0.3822-0.5258,0.146341,0.896825,0.346154,0.491071,0.346154,0.146341,0.205714,0.546,label=2
1,inception_v3,val,0.260870,0.539216,0.2276-0.8509,0.705882,0.600000,0.000000,0.260870,0.000000,0.705882,0.000000,0.506,label=2
2,inception_v3,test,0.404762,0.407407,0.2040-0.6108,0.958333,0.235294,0.000000,0.414634,0.000000,0.958333,0.000000,0.387,label=2
3,resnet101,train,0.648000,0.558959,0.4874-0.6305,0.154472,0.992126,0.786885,0.603175,0.786885,0.154472,0.258247,0.645,label=2
4,resnet101,val,0.521739,0.676471,0.4315-0.9215,0.588235,0.833333,0.800000,0.307692,0.800000,0.588235,0.677966,0.491,label=2
5,resnet101,test,0.357143,0.438657,0.2522-0.6251,0.291667,0.882353,0.000000,0.384615,0.000000,0.291667,0.000000,0.404,label=2
6,resnet50,train,0.528000,0.491902,0.4199-0.5639,0.829268,0.267717,0.560976,0.521531,0.560976,0.829268,0.669234,0.331,label=2
7,resnet50,val,0.260870,0.480392,0.2174-0.7434,0.176471,1.000000,0.000000,0.260870,0.000000,0.176471,0.000000,0.507,label=2
8,resnet50,test,0.428571,0.637731,0.4551-0.8203,0.750000,0.611111,0.000000,0.428571,0.000000,0.750000,0.000000,0.453,label=2
9,vgg19,train,0.508000,0.554862,0.4830-0.6267,0.276423,0.858268,0.000000,0.508000,0.000000,0.276423,0.000000,0.537,label=2
